In [1]:
import pandas as pd
import numpy as np
import os
import boto3
import sagemaker

from sklearn.preprocessing import StandardScaler

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


## Load Dasets

In [2]:
dataset = pd.read_csv("../datasets/merged_steam_games_.csv")


print("-------dataset------")
print(dataset.head())

-------dataset------
   app_id       title date_release  win  mac  linux  rating  positive_ratio  \
0  552520  Far Cry® 5   2018-03-26    1    0      0       7              80   
1  552520  Far Cry® 5   2018-03-26    1    0      0       7              80   
2  552520  Far Cry® 5   2018-03-26    1    0      0       7              80   
3  552520  Far Cry® 5   2018-03-26    1    0      0       7              80   
4  552520  Far Cry® 5   2018-03-26    1    0      0       7              80   

   user_reviews  price_final  ...  RPG  Indie  Software Training  Simulation  \
0        129943         60.0  ...    0      0                  0           0   
1        129943         60.0  ...    0      0                  0           0   
2        129943         60.0  ...    0      0                  0           0   
3        129943         60.0  ...    0      0                  0           0   
4        129943         60.0  ...    0      0                  0           0   

   Game Development Mas

Preprocessing

- Feature selection


In [3]:
# convert date_realese columns to unixtime
dataset['date_release'] = pd.to_datetime(dataset['date_release'])
dataset['date_release'] = dataset['date_release'].apply(lambda x: int(x.timestamp()))

dataset['date'] = pd.to_datetime(dataset['date'])
dataset['date'] = dataset['date'].apply(lambda x: int(x.timestamp()))

app_id_df = pd.DataFrame(dataset['app_id'])
# data = data.drop(['app_id'], axis=1)

titles_df = pd.DataFrame(dataset[['app_id', 'title']])
titles_df.drop_duplicates(['app_id'], keep='first', inplace=True, ignore_index=True)

# 전처리 안된 column 제거
dataset = dataset.drop(['categories'], axis=1)

dataset = dataset.drop(['title'], axis=1)
dataset = dataset.drop(['release_date'], axis=1)  

print(dataset.shape)
print(dataset.head())
print(dataset.columns)
print(dataset.dtypes)
print(dataset.describe())

(46365, 56)
   app_id  date_release  win  mac  linux  rating  positive_ratio  \
0  552520    1522022400    1    0      0       7              80   
1  552520    1522022400    1    0      0       7              80   
2  552520    1522022400    1    0      0       7              80   
3  552520    1522022400    1    0      0       7              80   
4  552520    1522022400    1    0      0       7              80   

   user_reviews  price_final  price_original  ...  RPG  Indie  \
0        129943         60.0             0.0  ...    0      0   
1        129943         60.0             0.0  ...    0      0   
2        129943         60.0             0.0  ...    0      0   
3        129943         60.0             0.0  ...    0      0   
4        129943         60.0             0.0  ...    0      0   

   Software Training  Simulation  Game Development  Massively Multiplayer  \
0                  0           0                 0                      0   
1                  0           0  

In [4]:
# Extract recommendation columns from dataset
recommend_columns = ['app_id','helpful', 'funny', 'date', 'is_recommended', 'hours',
       'user_id']

recommend_datas = dataset[recommend_columns]

game_infos = dataset.drop(recommend_columns[1:], axis=1)
scaled_game_info = game_infos.copy()
app_id_list = app_id_df['app_id'].unique()
app_id_indices = [app_id_df[app_id_df['app_id'] == value].index[0] for value in app_id_list]
print(game_infos.iloc[app_id_indices])

       app_id  date_release  win  mac  linux  rating  positive_ratio  \
0      552520    1522022400    1    0      0       7              80   
815    242760    1525046400    1    0      0       8              95   
2093   444200    1478649600    1    1      0       7              80   
2187   255710    1425945600    1    1      1       7              93   
2725      220    1191974400    1    1      1       8              97   
...       ...           ...  ...  ...    ...     ...             ...   
42719  427520    1597363200    1    1      1       8              96   
43254  814380    1553126400    1    0      0       8              95   
43938  945360    1542326400    1    0      0       7              92   
45411  477160    1469145600    1    1      0       7              94   
45819  250900    1415059200    1    1      1       8              97   

       user_reviews  price_final  price_original  ...  RPG  Indie  \
0            129943        60.00            0.00  ...    0      0 

## Scailing


In [5]:


scale_cols = ['date_release','positive_ratio','user_reviews','price_final','price_original', 'required_age', 'achievements', 'positive_ratings', 'negative_ratings', 
              'average_playtime', 'median_playtime', 'owners', 'price',]
# scale_cols = [col for col in scaled_data.columns if col not in no_scale_cols]


scaled_game_info = game_infos.iloc[app_id_indices]

# Scale the data
scaler = StandardScaler()
scaled_game_info.loc[:,scale_cols] = scaler.fit_transform(scaled_game_info.loc[:,scale_cols])

print(scaled_game_info.head())

      app_id  date_release  win  mac  linux  rating  positive_ratio  \
0     552520      0.729471    1    0      0       7       -1.339144   
815   242760      0.758106    1    0      0       8        0.611578   
2093  444200      0.318765    1    1      0       7       -1.339144   
2187  255710     -0.180300    1    1      1       7        0.351481   
2725     220     -2.395823    1    1      1       8        0.871674   

      user_reviews  price_final  price_original  ...  RPG  Indie  \
0        -0.360355     2.494874       -0.228311  ...    0      0   
815      -0.046625    -0.006719       -0.228311  ...    0      1   
2093     -0.358830    -1.257515       -0.228311  ...    0      0   
2187     -0.307168     0.618679       -0.228311  ...    0      0   
2725     -0.368811    -0.632742        2.842675  ...    0      0   

      Software Training  Simulation  Game Development  Massively Multiplayer  \
0                     0           0                 0                      0   
815 

## Deploy datasets

In [6]:
games = scaled_game_info.copy()
titles = titles_df.copy()
reviews = recommend_datas.copy()

In [7]:
%store games
%store titles
%store reviews

Stored 'games' (DataFrame)
Stored 'titles' (DataFrame)
Stored 'reviews' (DataFrame)


In [8]:
# get current session region
session = boto3.session.Session()
region = session.region_name
print(f'currently in {region}')

currently in ap-northeast-2


In [9]:
# use the default sagemaker s3 bucket to store processed data
# here we figure out what that default bucket name is 
sagemaker_session = sagemaker.Session()
bucket_name = 'sagemaker-gacheon-5'
print(bucket_name)
# bucket name format: "sagemaker-gacheon-{account 숫자}"
%store bucket_name

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker-gacheon-5
Stored 'bucket_name' (str)


In [12]:
# Specify the destination directory
dest = 'ml-latest-small/s3'

# Create the directory if it doesn't exist
os.makedirs(dest, exist_ok=True)

# Paths for saving the data
games_path = os.path.join(dest, 'games.npy')
titles_path = os.path.join(dest, 'titles.npy')
reviews_path = os.path.join(dest, 'reviews.npy')
games_columns_path = os.path.join(dest, 'games_columns.npy')
reviews_columns_path = os.path.join(dest, 'reviews_columns.npy')

# Save data locally
np.save(games_path, games.values, allow_pickle=True)
np.save(titles_path, titles.values, allow_pickle=True)
np.save(reviews_path, reviews.values, allow_pickle=True)
np.save(games_columns_path, games.columns, allow_pickle=True)
np.save(reviews_columns_path, reviews.columns, allow_pickle=True)

# Upload to S3 bucket (see the bucket name above)
sagemaker_session.upload_data(games_path, bucket=bucket_name, key_prefix='data')
sagemaker_session.upload_data(titles_path, bucket=bucket_name, key_prefix='data')
sagemaker_session.upload_data(reviews_path, bucket=bucket_name, key_prefix='data')
sagemaker_session.upload_data(games_columns_path, bucket=bucket_name, key_prefix='data')
sagemaker_session.upload_data(reviews_columns_path, bucket=bucket_name, key_prefix='data')


's3://sagemaker-gacheon-5/data/reviews_columns.npy'